In [105]:

from Experiment import Experiment
import matplotlib.pyplot as plt
import Plotters 
import plotly.graph_objects as go
import plotly
import numpy as np
import pandas as pd
import time
import matplotlib.cm as colormap

%matplotlib qt


loadir = 'J:/My Drive/dark 2022/csv_dark' # dir to save the data frame in # H:\My Drive\Ronis Exp\sagiv\data_analysis



exp_name =['manipulated_2022_02_03_dark','manipulated_2023_08_10_100ms','manipulated_2023_08_09_80ms','manipulated_2023_08_09_60ms','manipulated_2023_08_06_40ms','manipulated_12_8_23_blink20ms','manipulated_28_11_22_nopert','manipulated_11_8_23_blink40ms']
exp_name =['manipulated_2022_02_03_dark','manipulated_2023_08_10_100ms','manipulated_2023_08_09_80ms','manipulated_2023_08_09_60ms','manipulated_2023_08_06_40ms','manipulated_28_11_22_nopert']
# exp_name =['manipulated_12_8_23_blink20ms','manipulated_11_8_23_blink40ms','manipulated_13_8_23_blink60ms','manipulated_2022_03_03_220hz']

# exp_name =['manipulated_2023_08_09_60ms','manipulated_2022_02_03_dark']
# exp_name =['manipulated_28_11_22_nopert']

# ['manipulated_2023_08_07_10ms',
#             'manipulated_2023_08_06_40ms',
#            'manipulated_2023_08_09_60ms',
#            'manipulated_2023_08_09_80ms',
#            'manipulated_2023_08_10_100ms',
#            'manipulated_2022_02_03_dark',
#            'manipulated_2022_01_31_darkdown']

def dont_show_movs(exp_name,movs_to_del):
    return [mov_name for mov_name in experiments[exp_name].mov_names if mov_name not in movs_to_del ] 

mov_dark_delete = ['mov30','mov40','mov57','mov59']
mov_100_delete = ['mov10','mov102','mov19','mov34','mov35','mov47','mov51','mov52','mov66','mov68','mov73','mov90']
mov_80_delete = ['mov119','mov49','mov97','mov99']
mov_40_delete = ['mov64']
mov_60_delete = ['mov12','mov13','mov147','mov165','mov21','mov24','mov26','mov48','mov86']
mov_10_delete = ['mov46','mov54']
mov_5_delete = ['mov44']


color_map = colormap.datad["tab10"]['listed']

experiments = {exp:Experiment(loadir,exp) for exp in exp_name} # load experiments


In [106]:
# calculations - projections, frequency, angles

[exp.from_vector_to_wing_body('Y_x_body',['vectors','body'])  for exp in experiments.values()] # smooth and derive


[exp.smooth_prop_movies('CM_real_x_body',['','_dot','_dot_dot'],'body')  for exp in experiments.values()] # smooth and derive


[exp.project_axes_movies(['X_x_body_projected','X_y_body_projected'], add_to_vectors = ['vectors','body'])  for exp in experiments.values()] # X_body axis projected to xy plane
[exp.project_axes_movies(['Y_x_body_projected','Y_y_body_projected'],axis = 'Y_x_body',add_to_vectors = ['vectors','body'])  for exp in experiments.values()] # X_body axis projected to xy plane

[exp.project_axes_movies(['CM_dot_xax','CM_dot_yax'],axis = 'CM_real_x_body_dot_smth',wing_body = 'body',add_to_vectors = ['body','vectors'])  for exp in experiments.values()] # Velocity direction in the projected X_body axes (xy plane) 

# calculate the size of the velocity projected on the xy plane
[exp.project_prop_movies('CM_real_x_body_dot_smth',header_name = 'CM_dot_x',three_col = 2,ax_to_proj = 'X_x_body_projected',add_to_vectors = True)  for exp in experiments.values()]
[exp.project_prop_movies('CM_real_x_body_dot_smth',header_name = 'CM_dot_y',three_col = 2,ax_to_proj = 'Y_x_body_projected',add_to_vectors = True)  for exp in experiments.values()]


# calculate the size of the acceleration projected on the xy plane
[exp.project_prop_movies('CM_real_x_body_dot_dot_smth',header_name = 'CM_dot_dot_x',three_col = 2,ax_to_proj = 'X_x_body_projected')  for exp in experiments.values()]
[exp.project_prop_movies('CM_real_x_body_dot_dot_smth',header_name = 'CM_dot_dot_y',three_col = 2,ax_to_proj = 'Y_x_body_projected')  for exp in experiments.values()]

[exp.pqr_movies() for exp in experiments.values()] # calculate p,q,r (angular velocity in body axis)

[exp.mean_mean_props_movies('phi_rw','phi_lw','mean_wing','mean_mean_phi') for exp in experiments.values()] # calculate the mean value of two properties


[exp.substruct_first_frame('yaw_body','body')  for exp in experiments.values()] # substract the zero frame from a property ( {prop}_min_ref_frame )

# calculate the difference between current direction and another one at the same time:
# the angle between Xbody on the xy plane and the velocity direction on the xy plane 
[exp.delta_ang_all_time_movies('X_x_body_projected','CM_dot_xax','slip_angle_beta',three_col = 2) for exp in experiments.values()] 
[exp.delta_ang_all_time_movies('X_x_body_projected','CM_dot_xax','slip_angle_beta_no_uw',three_col = 2,unwarped = False) for exp in experiments.values()] 

# calculate the difference in angle between current property and in referance frame:
# the angle between Xbody on the xy (in 0 time) plane and Xbody on xy plane 
[exp.delta_ref_ang_movies('X_x_body_projected','delta_ang_Xax',axis = 'X_x_body_projected')  for exp in experiments.values()]

# the angle between Xbody on the xy (in 0 time) plane and the velocity direction on the xy plane 
[exp.delta_ref_ang_movies('CM_dot_xax','delta_ang_CM_dot_x',axis = 'CM_dot_xax',delta_ang = 0) for exp in experiments.values()]


# the angle between Xbody on the xy (in 0 time) plane and the velocity direction on the xy plane 
[exp.delta_ref_ang_movies('CM_dot_x_projected','delta_ang_proj_v',axis = 'CM_dot_x_projected',delta_ang = 0) for exp in experiments.values()]
[exp.delta_ref_ang_movies('CM_dot_x_projected','delta_ang_proj_v_uw',axis = 'CM_dot_x_projected',delta_ang = 0,unwarped = False) for exp in experiments.values()]



# project velocity and accelerations
[exp.project_prop_all_axes_movies('CM_real_x_body_dot_smth',header_name = 'CM_dot_x',three_col = 2,ax_to_proj = 'X_x_body_projected')  for exp in experiments.values()]
[exp.project_prop_all_axes_movies('CM_real_x_body_dot_smth',header_name = 'CM_dot_y',three_col = 2,ax_to_proj = 'Y_x_body_projected')  for exp in experiments.values()]

[exp.project_prop_all_axes_movies('CM_real_x_body_dot_dot_smth',header_name = 'CM_dot_dot_y',three_col = 2,ax_to_proj = 'Y_x_body_projected')  for exp in experiments.values()]
[exp.project_prop_all_axes_movies('CM_real_x_body_dot_dot_smth',header_name = 'CM_dot_dot_x',three_col = 2,ax_to_proj = 'X_x_body_projected')  for exp in experiments.values()]

[exp.project_prop_all_axes_movies('CM_real_x_body_dot_dot_smth',header_name = 'CM_dot_dot_y_body',three_col = 2,ax_to_proj = 'Y_x_body')  for exp in experiments.values()]

# calculate amplitude of phi and frequency
[exp.calculate_phi_amp_movies('rw','mean_body')  for exp in experiments.values()]
[exp.calculate_phi_amp_movies('lw','mean_body')  for exp in experiments.values()]

[exp.calculate_freq_movies('phi_rw_min_idx', 'mean_body')  for exp in experiments.values()]
[exp.calculate_freq_movies('phi_lw_min_idx', 'mean_body')  for exp in experiments.values()]


# averge properties by stroke 
props_to_mean = ['x','y','z']
[[exp.mean_by_stroke_movies(f'CM_real_{x}_body_dot_dot_smth','mean_body','body')  for exp in experiments.values()] for x in props_to_mean]
[[exp.mean_by_stroke_movies(f'CM_dot_dot_{x}_projected_all_axes','mean_body','body')  for exp in experiments.values()] for x in ['x','y']]
[[exp.mean_by_stroke_movies(f'CM_dot_x_projected','mean_body','body')  for exp in experiments.values()] for x in props_to_mean]

[[exp.mean_by_stroke_movies(f'X_{x}_body_projected','mean_body','body')  for exp in experiments.values()] for x in ['x','y']]
[[exp.mean_by_stroke_movies(f'Y_{x}_body_projected','mean_body','body')  for exp in experiments.values()] for x in ['x','y']]

# rotate for noams 3d traj plot
[exp.rotate_prop_movies('CM_real_x_body',['rotated_cm_x','rotated_cm_y','rotated_cm_z'],rotate_by_yaw = False) for exp in experiments.values()]

# histogram calculations
exp = experiments[list(experiments.keys())[0]]
[exp.get_peaks_movies('q','max') for exp in experiments.values()]
[exp.get_peaks_movies('CM_dot_dot_x_projected','min') for exp in experiments.values()]
[exp.get_peaks_movies('pitch_y_frame','max') for exp in experiments.values()]
[exp.get_peaks_movies('pitch_body','max') for exp in experiments.values()]
[exp.get_peaks_movies('CM_dot_x_projected','min') for exp in experiments.values()]
[exp.get_peaks_movies('delta_ang_Xax','max') for exp in experiments.values()]
[exp.get_peaks_movies('CM_dot_dot_x_projected','peaks_max',t0 = 5, t1 = 100) for exp in experiments.values()]
[exp.get_peaks_movies('CM_dot_x_projected','half_v') for exp in experiments.values()]

# [exp.subtract_interest_time_from_time('CM_dot_x_projected_zero_v','time_m_zero_v') for exp in experiments.values()]

# calculate velocity and acceleration norma
[[exp.norm_prop_movies(f'CM_real_{ax}_body_dot_smth',f'vel_norm')  for exp in experiments.values()] for ax in ['x']]
[[exp.norm_prop_movies(f'CM_real_{ax}_body_dot_dot_smth',f'acc_norm')  for exp in experiments.values()] for ax in ['x']]
[exp.norm_prop_movies(f'slip_angle_beta',f'slip_angle_beta_norm',three_col = 1)  for exp in experiments.values()] 

[exp.get_peaks_movies('acc_norm','respone_time',t0 = 15, t1 = 120,th = exp.th_for_response_time(th_mean = 1.9),th_mean = 1.9) for exp in experiments.values()]


t_vec = [50,60,70,80,90,100]
max_t = 90 # time to mesure the angle
acc_dir = [exp.acc_dir_movies( t,'CM_dot_dot_x_projected') for t in t_vec]
angle_z = np.arcsin(acc_dir[t_vec == max_t][:,2])*180/np.pi 
angle_x = np.arccos(-acc_dir[t_vec == max_t][:,0])*180/np.pi



[exp.get_peaks_movies('CM_dot_x_projected','zero_v',t0 = 15, t1 = 150) for exp in experiments.values()]
[exp.get_peaks_movies('CM_real_z_body_dot_smth','zero_v',t0 = 15, t1 = 150) for exp in experiments.values()]
[exp.calc_cone_angle_movies()  for exp in experiments.values()] 


# response time calculations: data that enters the analysis: mean of the acc[0: ref frame] > mean + std of mean of the acc[0: ref frame] (1.9)
#                             th for data that enters the anaylsis:  mean + std of mean of the acc[0: ref frame] that passed the previous th. 
#                             nan is - didnt pass the second. 999 is didnt enter the analysis

c:\Users\Roni\Documents\experiment-analysis\data_analysis\Movie.py:191: RuntimeWarning:

invalid value encountered in arccos

c:\Users\Roni\Documents\experiment-analysis\data_analysis\Movie.py:195: RuntimeWarning:

invalid value encountered in arccos

c:\Users\Roni\Documents\experiment-analysis\data_analysis\Movie.py:486: RuntimeWarning:

divide by zero encountered in divide



IndexError: index 0 is out of bounds for axis 0 with size 0

In [5]:
[exp.calc_cone_angle_movies()  for exp in experiments.values()] 


[None]

In [78]:
mov = exp.get_mov('mov101')
mov.get_prop('p','body',three_col=3)


array([[ 41.19037654,  88.84389595,  34.28248996],
       [ 40.68990637,  88.07202339,  34.42065769],
       [ 40.18862262,  87.28608603,  34.56070712],
       ...,
       [105.68910084, 131.36696106, -51.36815071],
       [104.72323174, 131.51387225, -51.03738052],
       [103.74132676, 131.66027871, -50.69943375]])

In [108]:
v

6.082765116075196

In [134]:
lstini = []
lstfin = []
# plt.figure()
for exp_name in experiments.keys(): 
    lstini = []

    exp = experiments[exp_name]
    for mov_name in exp.mov_names:
        mov = exp.get_mov(mov_name)
        v = mov.get_prop('CM_dot_x_projected','body',three_col=1)

        lstini.append(v[-200] - v[200])

        # plt.scatter(vnorm,pqrnorm)

    plt.hist(np.hstack(lstini),bins = 10,alpha = 0.3,label = exp_name)

plt.legend()

In [ ]:
exp = experiments['manipulated_28_11_22_nopert']
lst = []
for mov_name in exp.mov_names:
    mov = exp.get_mov(mov_name)
    v = mov.get_prop('CM_real_x_dot_body','body',three_col=3)

    vnorm = np.linalg(v)
    all = np.sum(np.sqrt((np.diff(cm,axis = 0)*100)**2))
    lst.append(all/path)
print(all/path)

plt.hist(np.hstack(lst),bins =20)


In [40]:
exp = experiments['manipulated_28_11_22_nopert']
lst = []
for mov_name in exp.mov_names:
    mov = exp.get_mov(mov_name)
    cm = mov.get_prop('CM_real_x_body','body',three_col=3)

    path = np.sum(np.sqrt((cm[0,:] - cm[-1,:])**2))*100
    all = np.sum(np.sqrt((np.diff(cm,axis = 0)*100)**2))
    lst.append(all/path)
print(all/path)

plt.hist(np.hstack(lst),bins =20)


1.4924212269685548


(array([14., 36., 37., 35., 30., 12.,  5.,  5.,  4.,  2.,  3.,  1.,  1.,
         1.,  2.,  2.,  0.,  1.,  0.,  1.]),
 array([1.10040869, 1.19884743, 1.29728617, 1.39572491, 1.49416364,
        1.59260238, 1.69104112, 1.78947986, 1.8879186 , 1.98635734,
        2.08479607, 2.18323481, 2.28167355, 2.38011229, 2.47855103,
        2.57698977, 2.6754285 , 2.77386724, 2.87230598, 2.97074472,
        3.06918346]),
 <BarContainer object of 20 artists>)

In [32]:
np.diff(cm,axis = 0)**2

np.sqrt(np.sum((cm[0] - cm[1])**2))

6.554218972053433e-06

In [ ]:
# CSV FOR NOAM
path = 'I:/.shortcut-targets-by-id/1OA70vOJHDfV63DqG7LJCifTwW1h055ny/2024 Flight in the dark paper/data_exchange/fly/'
file_name_model = 'model_fly'
file_name_histogram = 'zero_vel_x_z_and_respone_time.csv'
file_name_frequency = 'frequency_'
file_name_acc = 'acceleration_direction_time'
mov_name = ['mov11','mov70']
exp = ['manipulated_2022_02_03_dark','manipulated_2023_08_09_60ms']

# model ##########################################################

def generate_model_csv(mov_name,exp_name,file_name_model):
    exp = experiments[exp_name]
    mov = exp.get_mov(mov_name)
    props_3col = ['CM_real_x_body','pitch_body']
    props_1col = ['CM_real_x_body','CM_real_y_body','CM_real_z_body','yaw_body','pitch_body','roll_body','CM_dot_x_projected','CM_dot_dot_x_projected','time']
    exp_name = exp.experiment_name.split('_')[-1]

    header = ['x','y','z','yaw','pitch','roll','forward-vel','forward-acc','time']

    data_model = np.hstack([mov.get_prop(prop,'body',three_col=1) for prop in props_1col])
    df = pd.DataFrame(data_model, columns = header).to_csv(f'{path}{file_name_model}_{mov_name}_{exp_name}.csv',index=False)

[generate_model_csv(mov_name,exp,file_name_model) for mov_name,exp in zip(mov_name,exp)]


# response time #######################################################

response_time = np.hstack([exp.interest_points['acc_norm_respone_time'][:,exp.body_header['time']] for exp in experiments.values()])
zero_v = np.hstack([exp.interest_points['CM_dot_x_projected_zero_v'][:,exp.body_header['time']] for exp in experiments.values()])
zero_z = np.hstack([exp.interest_points['CM_real_z_body_dot_smth_zero_v'][:,exp.body_header['time']] for exp in experiments.values()])
header = ['zero_vel_z','zero_vel_x','response_time']
pd.DataFrame(np.vstack([zero_z,zero_v,response_time]).T, columns = header).to_csv(path + file_name_histogram,index=False)

# frequency ##########################################################
t_vec = np.linspace(0.79, 207.5, num=10)
header = [f'{int(t)} [ms]' for t in t_vec]
delta_t = 73*5/16000*1000
prop = 'freq_phi_rw_min_idx'
data = [exp.mean_prop_time_vector_movies(prop,delta_t,t_vec, mov = False,get_delta = 0) for exp in  experiments.values()]

for idx,exp_name in enumerate(experiments.keys()):
    exp_name = exp_name.split('_')[-1]
    exp_freq = np.vstack([data[idx]['freq_phi_rw_min_idx'][data[idx]['t0']== t] for t  in  t_vec]).T
    pd.DataFrame(exp_freq, columns = header).to_csv(path + file_name_frequency + exp_name + '.csv',index=False)


# acceleration direction ###########################################################

max_t = 90 # time to mesure the angle
acc_dir = np.vstack([exp.acc_dir_movies(max_t,'CM_real_x_body_dot_dot_smth') for exp in experiments.values()])
pd.DataFrame(acc_dir, columns = ['acc_x','acc_y','acc_z']).to_csv(f'{path}{file_name_acc}_{max_t}ms.csv',index=False)






In [ ]:
# all data for violin plots
import pandas as pd
path = 'I:/.shortcut-targets-by-id/1OA70vOJHDfV63DqG7LJCifTwW1h055ny/2024 Flight in the dark paper/data_exchange/fly/all_data/'

props_1d = ['time','yaw_body','pitch_body','CM_real_z_body_dot_smth','CM_dot_x_projected','slip_angle_beta','delta_ang_proj_v','slip_angle_beta_no_uw','delta_ang_proj_v_uw','cone_angle']
header =['mov_num','time','yaw','pitch','z_vel','forward_vel','beta','vel_xy_ang','beta_flat','vel_xy_ang_flat','cone_angle']


# props_1d = ['time','yaw_body','pitch_body','pitch_body_dot','pitch_body_dot_dot','CM_real_z_body_dot_smth',
#             'CM_dot_x_projected','CM_dot_y_projected','slip_angle_beta','delta_ang_proj_v']
# header =['mov_num','time','yaw','pitch','pitch_dot','pitch_dot_dot','z_vel','forward_vel','velocity_y','beta','vel_xy_ang']


for exp in experiments.values():
    exp_name = exp.experiment_name.split('_')[-1]
    movs = []
    if 'dark' in exp_name:
        exp_name = 'step'

    for idx,mov_name in enumerate(exp.mov_names):
        mov = exp.get_mov(mov_name)
        props = [mov.get_prop(prop,'body') for prop in props_1d ]
        movs.append(np.hstack((props[0]*0+int(mov_name.split('mov')[-1]),np.hstack(props))))

    pd.DataFrame(np.vstack(movs), columns = header).to_csv(f'{path}{exp_name}_all_data.csv',index=False)


In [ ]:
# NOMINAL CALCULATIONS
exp.calculate_phi_amp_movies( 'rw','mean_wing')

exp = experiments['manipulated_28_11_22_nopert']
path = 'I:/.shortcut-targets-by-id/1OA70vOJHDfV63DqG7LJCifTwW1h055ny/2024 Flight in the dark paper/data_exchange/fly/'


 
# [[exp.norm_prop_movies(f'CM_real_{ax}_body_dot_smth',f'vel_norm')  for exp in experiments.values()] for ax in ['x']]
# [[exp.norm_prop_movies(f'CM_real_{ax}_body_dot_dot_smth',f'acc_norm')  for exp in experiments.values()] for ax in ['x']]
# [[exp.norm_prop_movies(f'slip_angle_beta',f'slip_angle_beta_norm')  for exp in experiments.values()] for ax in ['x']]

# slip angle and v
props_dict = {prop : np.vstack(exp.get_allexp_props(prop,wing_body = 'body')) for prop in 
              ['slip_angle_beta_norm','slip_angle_beta','CM_dot_x_projected','CM_real_z_body_dot_smth','CM_dot_y_projected','pitch_body','yaw_body','roll_body','p','q','r','cone_angle','vel_norm']}


# mean and std of pitch, velocity norma, acceleration normal and slip angle normal
mean_dict = {prop : np.nanmean(np.vstack(exp.get_allexp_props(prop,wing_body = 'body'))) for prop in ['pitch_body','vel_norm','acc_norm','slip_angle_beta_norm']}
std_dict = {prop : np.nanstd(np.vstack(exp.get_allexp_props(prop,wing_body = 'body'))) for prop in ['pitch_body','vel_norm','acc_norm','slip_angle_beta_norm']}
print(f'mean {mean_dict}')
print(f'std {std_dict}')
# frequency
mean_dict = {prop : np.nanmean(np.vstack(exp.get_allexp_props(prop,wing_body = 'mean_body'))) for prop in ['freq_phi_rw_min_idx']}
std_dict = {prop : np.nanstd(np.vstack(exp.get_allexp_props(prop,wing_body = 'mean_body'))) for prop in ['freq_phi_rw_min_idx']}

print(f'mean {mean_dict}')
print(f'std {std_dict}')
# wing_amp
mean_dict = {prop : np.nanmean(np.vstack(exp.get_allexp_props(prop,wing_body = 'mean_wing'))) for prop in ['amp_rw']}
std_dict = {prop : np.nanstd(np.vstack(exp.get_allexp_props(prop,wing_body = 'mean_wing'))) for prop in ['amp_rw']}



print(f'mean {mean_dict}')
print(f'std {std_dict}')
# histograms ################
# frequency
mean_dict = {prop : np.nanmean(np.vstack(exp.get_allexp_props(prop,wing_body = 'body'))) for prop in ['slip_angle_beta_norm']}
std_dict = {prop : np.nanstd(np.vstack(exp.get_allexp_props(prop,wing_body = 'body'))) for prop in ['slip_angle_beta_norm']}
print(f'mean {mean_dict}')
print(f'std {std_dict}')
Plotters.matplotlib_hist(props_dict['slip_angle_beta_norm'],'beta slip angle [deg]',f'beta slip angle [deg]',bins = 40)

# projected velocity in X direction
CM_dot_x_dict = {prop : np.vstack(exp.get_allexp_props(prop,wing_body = 'body')) for prop in ['CM_dot_x_projected']}
Plotters.matplotlib_hist(CM_dot_x_dict['CM_dot_x_projected'],'beta slip angle [deg]',f'CM_dot_x_projected',bins = 40)

# slip angle vs projected velocity in X direction
plt.figure()
plt.plot(CM_dot_x_dict['CM_dot_x_projected'],props_dict['slip_angle_beta_norm'])

# velocity in Z direction
prop = 'CM_real_z_body_dot_smth'
cmz = {prop : np.vstack(exp.get_allexp_props(prop,wing_body = 'body')) }
Plotters.matplotlib_hist(cmz[prop],'time [ms]',f'z vel',bins = 15)

# angle of the velocity direction
ang = []
for mov_name in exp.mov_names:
    mov = exp.get_mov(mov_name)
    vel = mov.get_prop('CM_real_x_body_dot_smth',wing_body='body',three_col=3)
    norm_vel = vel/np.linalg.norm(vel,axis = 1, keepdims=True)
    ang.append(np.arcsin(norm_vel[:,2])*180/np.pi)



header = ['vel_xy_ang','beta_norm','beta','forward_vel','z_vel','side_vel','pitch','yaw','roll','p','q','r','cone_angle','vel_norm','vel_dir_x','vel_dir_y','vel_dir_z']
vel_dir = exp.vel_dir_movies('CM_real_x_body_dot_smth') 

Plotters.matplotlib_hist(np.hstack(ang),'angle [deg]',f'nominal velocity angle',bins = 15)
pd.DataFrame(np.vstack((np.hstack(ang),np.hstack(props_dict.values()).T,vel_dir.T)).T, columns = header).to_csv(f'{path}v_ang_beta_nominal.csv',index=False)


mov_name = 'mov100'
exp = experiments['manipulated_28_11_22_nopert']
mov = exp.get_mov(mov_name)
wings_angle = np.hstack([mov.get_prop(wing_angle,wing_body='wing')[mov.ref_frame:mov.ref_frame + 73*4] for wing_angle in ['phi_lw','psi_lw','theta_lw']])
pd.DataFrame(wings_angle, columns = ['phi_lw','psi_lw','theta_lw']).to_csv(f'{path}wing_angles.csv',index=False)


# pd.DataFrame(vel_dir, columns = ['acc_x','acc_y','acc_z']).to_csv(f'{path}{file_name_vel}.csv',index=False)


# pd.DataFrame(acc_dir, columns = ['acc_x','acc_y','acc_z']).to_csv(f'{path}{file_name_acc}_{max_t}ms.csv',index=False)



In [6]:

ang_lst = []
exp_nom = []
for exp in list(experiments.values()):    
     for mov_name in exp.mov_names:
       mov = exp.get_mov(mov_name)
       projected_v = mov.get_prop('X_x_body_projected',three_col=2,wing_body = 'body')
       vector_v = mov.get_prop('CM_real_x_body_dot_smth',three_col=3,wing_body = 'body')
       proj_v = np.hstack((projected_v,projected_v[:,0:1]*0))

       vector_v2 = vector_v.T/np.linalg.norm(vector_v.T,axis = 0)
       proj_v2 = proj_v.T/np.linalg.norm(proj_v.T,axis = 0)
       ang = np.arccos(np.sum(vector_v2.T*proj_v2.T,axis = 1))*180/np.pi
       if exp.experiment_name == 'manipulated_28_11_22_nopert':
        exp_nom.append(ang)
       else:
        ang_lst.append(ang)



plt.hist(np.hstack(ang_lst),bins = 30)
plt.hist(np.hstack(exp_nom),bins = 30)

ValueError: need at least one array to concatenate

In [7]:
prop_at_zero = []
plt.figure()
for exp in list(experiments.values()):    

    for mov_name in exp.mov_names:
        mov = exp.get_mov(mov_name)
        time = mov.get_prop('time',wing_body='body')

        prop = mov.get_prop('acc_norm',wing_body='body')

        prop_at_zero.append(prop[mov.ref_frame])
        if np.mean(prop[0:mov.ref_frame]) < 1.5:
            plt.plot(time,prop)

plt.hist(prop_at_zero)
plt.hlines(1.3,0,350,colors=['black'])

In [24]:
plt.hist(np.hstack(prop_at_zero))
plt.vlines(1.3,0,80,colors=['black'])

In [10]:
prop = []
for exp in list(experiments.values()):    
    for mov_name in exp.mov_names:
        mov = exp.get_mov(mov_name)
        prop.append(np.nanmean(mov.get_prop('acc_norm',wing_body='body')[0:mov.ref_frame]))

print(np.mean(prop) + np.std(prop))

1.5661593710660875


In [7]:
path = 'I:/.shortcut-targets-by-id/1OA70vOJHDfV63DqG7LJCifTwW1h055ny/2024 Flight in the dark paper/data_exchange/fly/'
file_name_acc = 'velocity_direction'
exp = experiments['manipulated_28_11_22_nopert']
max_t = 90 # time to mesure the angle


mov = exp.get_mov('mov100')


acc_dir = np.hstack([mov.acc_dir(t,'CM_real_x_body_dot_dot_smth') for t in mov.get_prop('time','body')])
# pd.DataFrame(acc_dir, columns = ['acc_x','acc_y','acc_z']).to_csv(f'{path}{file_name_acc}_{max_t}ms.csv',index=False)



[array([-0.13527889, -0.93845107, -0.31781947]),
 array([-0.13511049, -0.93913893, -0.31585317]),
 array([-0.13494078, -0.93982445, -0.31388054]),
 array([-0.13476975, -0.94050757, -0.31190163]),
 array([-0.1345974 , -0.94118824, -0.3099165 ]),
 array([-0.13442371, -0.94186641, -0.30792522]),
 array([-0.13424868, -0.94254202, -0.30592784]),
 array([-0.13407228, -0.94321502, -0.30392442]),
 array([-0.13389452, -0.94388536, -0.30191503]),
 array([-0.13371537, -0.94455299, -0.29989973]),
 array([-0.13353482, -0.94521786, -0.29787858]),
 array([-0.13335287, -0.94587992, -0.29585164]),
 array([-0.1331695 , -0.94653911, -0.29381898]),
 array([-0.13298469, -0.9471954 , -0.29178066]),
 array([-0.13279844, -0.94784872, -0.28973674]),
 array([-0.13261073, -0.94849903, -0.28768729]),
 array([-0.13242154, -0.94914629, -0.28563237]),
 array([-0.13223087, -0.94979044, -0.28357206]),
 array([-0.1320387 , -0.95043144, -0.28150641]),
 array([-0.13184501, -0.95106924, -0.27943549]),
 array([-0.13164979,

In [10]:
mean_dict = {prop : np.nanmean(np.vstack(exp.get_allexp_props(prop,wing_body = 'body'))) for prop in ['roll_body','yaw_body']}
std_dict = {prop : np.nanstd(np.vstack(exp.get_allexp_props(prop,wing_body = 'body'))) for prop in ['roll_body','yaw_body']}

print(mean_dict,std_dict)

{'roll_body': -0.48196374747386256, 'yaw_body': 178.44972909588066} {'roll_body': 8.88253042384142, 'yaw_body': 103.2702598242453}


In [4]:
vel_dir = exp.vel_dir_movies('CM_real_x_body_dot_smth') 


c:\Users\Roni\anaconda3\lib\site-packages\numpy\core\shape_base.py:121: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.



0.08279919624328613
0.09292078018188477
0.09366226196289062
0.06172966957092285
0.07416796684265137
0.04739022254943848
0.08029937744140625
0.07407784461975098
0.08661270141601562
0.0754079818725586
0.07458329200744629
0.08689641952514648
0.07897710800170898
0.06666445732116699
0.08110213279724121
0.06429147720336914
0.06239938735961914
0.1179506778717041
0.06191754341125488
0.0582273006439209
0.11653828620910645
0.16383099555969238
0.08778905868530273
0.1056206226348877
0.10431170463562012
0.10045886039733887
0.08460259437561035
0.0868387222290039
0.07840967178344727
0.06682372093200684
0.0834648609161377
0.07260560989379883
0.057704925537109375
0.08693766593933105
0.08179855346679688
0.07965469360351562
0.0744926929473877
0.05270552635192871
0.08363580703735352
0.08569622039794922
0.07405662536621094
0.08068108558654785
0.11996650695800781
0.09435057640075684
0.10226583480834961
0.14245176315307617
0.15125489234924316
0.08498764038085938
0.1239008903503418
0.10444164276123047
0.09967

In [9]:

vel_dir = []
for mov_name in exp.mov_names:
    mov = exp.get_mov(mov_name)
    vel_dir.append([mov.acc_dir(t,'CM_real_x_body_dot_dot_smth') for t in mov.get_prop('time','body')])


In [4]:

mov_name = 'mov100'
exp = experiments['manipulated_28_11_22_nopert']
mov = exp.get_mov(mov_name)
mov.get_prop('slip_angle_beta_norm','body')

array([[5.45757936e+01],
       [5.51031405e+01],
       [5.59109749e+01],
       ...,
       [2.48883531e-01],
       [5.18990747e-02],
       [2.66136209e-01]])

In [93]:
np.linalg.norm(props_dict['slip_angle_beta'],axis = 1)

array([54.57579365, 55.10314055, 55.91097487, ..., 70.33394422,
       69.8875667 , 69.53573229])

In [87]:
plt.plot(props_dict['slip_angle_beta'])
plt.plot(props_dict['slip_angle_beta_norm'])

props_dict['slip_angle_beta']
np.linalg.norm(props_dict['slip_angle_beta'])

nan

In [74]:
plt.hist(props_dict['slip_angle_beta_norm'])

(array([5.87892e+05, 1.39122e+05, 3.40610e+04, 7.17400e+03, 7.97000e+02,
        6.29000e+02, 4.66000e+02, 4.95000e+02, 5.94000e+02, 1.96800e+03]),
 array([4.88622139e-03, 3.62973268e+01, 7.25897674e+01, 1.08882208e+02,
        1.45174649e+02, 1.81467089e+02, 2.17759530e+02, 2.54051970e+02,
        2.90344411e+02, 3.26636851e+02, 3.62929292e+02]),
 <BarContainer object of 10 artists>)

In [9]:
# frequency plot
t_vec = np.linspace(0.79, 207.5, num=10)
delta_t = 73*5/16000*1000
prop = 'freq_phi_rw_min_idx'
data = [exp.mean_prop_time_vector_movies(prop,delta_t,t_vec, mov = False,get_delta = 1) for exp in  experiments.values()]

# stats.ttest_ind()
for idx,exp_name in enumerate(experiments.keys()):
    mean = [np.nanmean(data[idx]['freq_phi_rw_min_idx'][data[idx]['t0']== t]) for t  in  t_vec]
    std = [np.nanstd(data[idx]['freq_phi_rw_min_idx'][data[idx]['t0']== t]) for t  in  t_vec]

    plt.errorbar(t_vec, mean, std, linestyle='-', marker='^',label = exp_name)
plt.legend()

d:\Documents\experiment-analysis\data_analysis\Movie.py:424: RuntimeWarning:

Mean of empty slice

d:\Documents\experiment-analysis\data_analysis\Movie.py:424: RuntimeWarning:

Mean of empty slice

d:\Documents\experiment-analysis\data_analysis\Movie.py:424: RuntimeWarning:

Mean of empty slice

d:\Documents\experiment-analysis\data_analysis\Movie.py:424: RuntimeWarning:

Mean of empty slice

d:\Documents\experiment-analysis\data_analysis\Movie.py:424: RuntimeWarning:

Mean of empty slice

C:\Users\Roni\AppData\Local\Temp\ipykernel_8556\1263881124.py:9: RuntimeWarning:

Mean of empty slice

c:\Users\Roni\anaconda3\lib\site-packages\numpy\lib\nanfunctions.py:1878: RuntimeWarning:

Degrees of freedom <= 0 for slice.



In [13]:
[exp.substruct_first_frame('pitch_body','body')  for exp in experiments.values()] # substract the zero frame from a property ( {prop}_min_ref_frame )


[None, None, None, None]

In [137]:


# for exp in experiments.values():
#     # plot slip angle and delta yaw
#     fig = go.Figure()
#     exp.plot_prop_movies('slip_angle_beta','body',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None)
#     fig.update_layout(title = exp.experiment_name)
#     fig.show()
pert = [20,40,60,1/220*1000]
clr = ['tomato','blue']
for exp_idx,exp in enumerate(experiments.values()):

    # pitch, roll corellations - plot pitch and roll vs acceleration
    fig = go.Figure()
    exp.plot_prop_movies('CM_dot_dot_y_projected_all_axes','body',fig,case = 'plot_mov',add_horizontal_line=None)
    [fig.add_vrect(x0=(0 + idx)*pert[exp_idx] , x1=(0 + idx)*pert[exp_idx] + pert[exp_idx], fillcolor=clr[np.mod(idx,2)], opacity=0.2) for idx in range(20)]
    fig.update_layout(title = exp.experiment_name)
    
    fig.show()

# for exp in experiments.values():

#     # pitch, roll corellations - plot pitch and roll vs acceleration
#     fig = go.Figure()
#     exp.plot_prop_movies('CM_dot_dot_x_projected_all_axes','body',fig,case = 'plot_mov',add_horizontal_line=None)
#     fig.update_layout(title = exp.experiment_name)
#     fig.show()

# for exp in experiments.values():

#     # pitch, roll corellations - plot pitch and roll vs acceleration
#     fig = go.Figure()
#     exp.plot_prop_movies('pitch_body','body',fig,case = 'plot_mov',add_horizontal_line=None)
#     fig.update_layout(title = exp.experiment_name)
#     fig.show()



In [6]:
exp.plot_prop_movies('p','body',fig,case = 'plot_mov',add_horizontal_line=None)


NameError: name 'fig' is not defined

In [136]:
# PLOTS

exp = experiments['manipulated_11_8_23_blink40ms']

# z velocity
fig = go.Figure()
exp.plot_prop_movies('CM_real_z_body_dot_smth','body',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None)
fig.show()

# pitch, roll corellations - plot pitch and roll vs acceleration
fig = go.Figure()
exp.plot_prop_movies('pitch_body','body',fig,case = 'plot_mov',prop_x = 'CM_dot_dot_x_projected_all_axes' ,add_horizontal_line=None,name = 'calculated')
fig.show()

fig = go.Figure()
exp.plot_prop_movies('roll_body','body',fig,case = 'plot_mov',prop_x = 'CM_dot_dot_y_projected_all_axes' ,add_horizontal_line=None,name = 'calculated')
fig.show()

# plot pitch and roll vs acceleration
fig = go.Figure()
exp.plot_prop_movies('pitch_body','body',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None)
fig.show()


# plot slip angle and delta yaw
fig = go.Figure()
exp.plot_prop_movies('slip_angle_beta','body',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None)
fig.show()


# delta angle projected velocity 
fig = go.Figure()
exp.plot_prop_movies('delta_ang_proj_v','body',fig,case = 'plot_mov',prop_x = 'time' ,add_horizontal_line=None)
fig.show()

# delta yaw
fig = go.Figure()
prop = 'delta_ang_Xax'
[exp.plot_prop_movies(prop,'body',fig,case = 'plot_exp',prop_x = 'time',color = color_map[idx],legend = exp.pertubation_name) for idx,exp in enumerate(experiments.values())]
fig.show()




# response time hisotgram
min_time = exp.interest_points['acc_norm_respone_time'][:,exp.body_header['time']]
Plotters.matplotlib_hist(min_time,'time [ms]','response tims [ms]',bins = 20)

# zero v xy
min_time = exp.interest_points['CM_dot_x_projected_zero_v'][:,exp.body_header['time']]
prec = 1-np.sum(np.isnan(min_time))/len(min_time)
Plotters.matplotlib_hist(min_time,'time [ms]',f'v_x_y = 0 prec = {prec}',bins = 15)


# zero v z
min_time = exp.interest_points['CM_real_z_body_dot_smth_zero_v'][:,exp.body_header['time']]
prec = 1-np.sum(np.isnan(min_time))/len(min_time)
Plotters.matplotlib_hist(min_time,'time [ms]',f'v_z = 0 prec = {prec}',bins = 15)

# sphere angle
Plotters.matplotlib_hist(angle_z,'xz angle','acceleration xy angle',bins = 20)

# sphere
fig = go.Figure()
[Plotters.only_scatter3d(fig,acc_dir[idx], f'time: {t} [ms]',mode = 'markers',marker=dict(size=3)) for idx,t in enumerate(t_vec)]
fig.add_trace(go.Scatter3d(x=[0],y=[0],z=[0],marker=dict(size=15),mode = 'markers'))
fig.show()

# # brake hisotgram
# point_name = 'CM_dot_x_projected_zero_v'
# list_trace,layout = zip(*[exp.interest_point_hist(point_name,xbins = dict(start=0,end=300, size=5)) for exp in experiments.values()])
# fig = go.Figure(data=list_trace, layout=layout[0]).show()

# 3d trajectory
exp = experiments['manipulated_2022_01_31_darkdown']
mov = ['mov101']
exp.plot_3d_traj_movies('CM_dot_x_projected',mov = mov)
fig.show()

# noams 3d traj plot 
fig = go.Figure()
[Plotters.only_scatter3d(fig,exp.get_mov(mov_name).get_prop('rotated_cm_x','body',three_col=3)*1000, mov_name,skip = 400,marker=dict(size=3)) for mov_name in exp.mov_names]
fig.show()

# frequency plot
t_vec = list(range(0,130,10))
delta_t = 73*5/16000*1000
prop = 'freq_phi_rw_min_idx'
data = [exp.mean_prop_time_vector_movies(prop,delta_t,t_vec, mov = False,get_delta = 1) for exp in  experiments.values()]

# stats.ttest_ind()
for idx,exp_name in enumerate(experiments.keys()):
    mean = [np.nanmean(data[idx]['freq_phi_rw_min_idx'][data[idx]['t0']== t]) for t  in  t_vec]
    std = [np.nanstd(data[idx]['freq_phi_rw_min_idx'][data[idx]['t0']== t]) for t  in  t_vec]

    plt.errorbar(t_vec, mean, std, linestyle='-', marker='^',label = exp_name)
plt.legend()

# plot wings angle
# exp = experiments['manipulated_2023_08_09_60ms']

# phi_r_color = [[256,0,0]]*len(exp.mov_names)
# phi_l_color = [[0,0,256]]*len(exp.mov_names)

# fig = go.Figure()
# exp.plot_prop_movies('theta_rw','wing',fig,case = 'plot_mov',prop_x = 'time',color = phi_r_color )
# exp.plot_prop_movies('theta_lw','wing',fig,case = 'plot_mov',prop_x = 'time', color = phi_l_color)
# fig.show()

# fig = go.Figure()
# exp.plot_prop_movies('psi_rw','wing',fig,case = 'plot_mov',prop_x = 'time',color = phi_r_color )
# exp.plot_prop_movies('psi_lw','wing',fig,case = 'plot_mov',prop_x = 'time', color = phi_l_color)
# fig.show()


# fig = go.Figure()
# exp.plot_prop_movies('phi_lw','wing',fig,case = 'plot_mov',prop_x = 'time',color = phi_l_color )
# exp.plot_prop_movies('phi_rw','wing',fig,case = 'plot_mov',prop_x = 'time', color = phi_r_color)

# fig.show()






KeyError: 'manipulated_2022_01_31_darkdown'

In [ ]:
prop_name = 'delta_ang_proj_v'
for exp in experiments.values():
    for mov_name in exp.mov_names: 
        mov = exp.get_mov(mov_name)
        prop = np.abs(mov.get_prop(prop_name,'body'))
        mov.data['body'] = np.hstack((mov.data['body'], prop))
        mov.add_to_header( ['delta_ang_proj_v_abs'],'body')

In [ ]:

# violin fly mosq
import plotly.graph_objects as go


t_vec = time_vec
delta_t = 73*5/16000*1000
prop = 'slip_angle_beta_no_uw'



fig = go.Figure()

for tfin in t_vec:
    for exp in  experiments.values():

        delta_v = np.vstack(np.abs(exp.get_prop_on_time_movies(prop,'body', time_prop = 'time', t_fin = tfin, delta_frames = 5)))
            # Creating violin plot using graph objects
        fig.add_trace(go.Violin(x=delta_v[:,0]*0+tfin, y=delta_v[:,0],
                                box_visible=False,
                                name = f'{tfin}', 
                                legendgroup = exp.experiment_name,
                                ))
fig.update_layout(xaxis_title=prop,yaxis_title='time')
fig.add_hline(y=180)
            
fig.show()

    

import pandas as pd
path = 'J:/My Drive/dark 2022/noams_csv/'
pulse = '60ms_pulse_beta _angle.csv'
dark = 'step_beta_angle.csv'
pulse = pd.read_csv(f'{path}{pulse}',names=["beta", "time"])
dark = pd.read_csv(f'{path}{dark}',names=["beta", "time"])

time_vec = np.unique(pulse['time'])
time_vec


fig = go.Figure()

for tfin in time_vec:
    for exp,experiment_name in  zip([pulse,dark],['pulse','dark']):
        
        
        delta_v = np.array(exp)[exp['time'] == tfin,0]
            # Creating violin plot using graph objects
        fig.add_trace(go.Violin(x=delta_v*0+tfin, y=delta_v,
                                box_visible=False,
                                name = f'{tfin}_{experiment_name}', 
                                legendgroup = experiment_name,
                                ))
fig.update_layout(xaxis_title=prop,yaxis_title='time')
fig.add_hline(y=180)

            
fig.show()


array([0.02562734, 0.02565886, 0.02572067, ..., 0.02772449, 0.02777529,
       0.02749803])

In [87]:
# [exp.substruct_first_frame('yaw_body','body')  for exp in experiments.values()] # substract the zero frame from a property ( {prop}_min_ref_frame )

def get_fft(prop,mov,sampling_rate = 16000):

    signal  = mov.get_prop(prop,wing_body='body').T[0]
    mask = np.isfinite(signal)
    x = signal
    xi = np.arange(len(signal))

    signal = np.interp(xi, xi[mask], x[mask])

    
    # Step 2: Compute the FFT
    fft_result = np.fft.fft(signal)
    fft_freq = np.fft.fftfreq(len(signal), 1/sampling_rate)

    # Step 3: Compute the amplitude spectrum
    amplitude_spectrum = np.abs(fft_result)
    return fft_freq,amplitude_spectrum


exp = experiments['manipulated_13_8_23_blink60ms']
exp = experiments['manipulated_11_8_23_blink40ms']
exp = experiments['manipulated_12_8_23_blink20ms']

prop = 'CM_dot_dot_x_projected_all_axes'
fft_exp = [get_fft(prop,exp.get_mov(mov_name)) for mov_name in exp.mov_names]



In [138]:
import matplotlib.pyplot as plt
import numpy as np

# Example data structure (to be replaced with your actual data):
# fft_exp = [ [frequencies_1, amplitudes_1], [frequencies_2, amplitudes_2], ... ]
# Example: fft_exp = [(np.linspace(0, 100, 100), np.abs(np.random.randn(100))) for _ in range(5)]


exp = experiments['manipulated_13_8_23_blink60ms']
for exp_idx,exp in enumerate(experiments.values()):
    plt.figure()

    prop = 'CM_dot_dot_y_projected_all_axes'
    prop = 'pitch_body_dot'

    fft_exp = [get_fft(prop,exp.get_mov(mov_name)) for mov_name in exp.mov_names]


    # Adjust the number of plots
    num_plots = len(fft_exp)

    # Generate colormap values
    colormap = plt.cm.viridis
    colors = colormap(np.linspace(0, 1, num_plots))

    # Create a figure with a fixed size

    for idx, fft_all in enumerate(fft_exp):
        # Plot the amplitude spectrum with different colors for each plot
        # markerline, stemlines, baseline = plt.stem(1/fft_all[0]*1000, fft_all[1], basefmt=" ", use_line_collection=True)
        x = fft_all[1][fft_all[1] >= 0 ]
        y = fft_all[0][fft_all[1] >= 0 ]
        plt.plot(1/fft_all[0]*1000, fft_all[1],colors[exp_idx])
        # Set colors for the lines and markers
        # plt.setp(stemlines, color=colors[idx])
        # plt.setp(markerline, color=colors[idx])

    # Customize the plot
    plt.title('Amplitude Spectrum')
    plt.xlabel('period (ms)')
    plt.ylabel('Amplitude')
    plt.xlim(0, 350)  # Focus on positive frequencies
    plt.grid(True)
    # plt.yscale('log')

    # Show the plot
    plt.show()
    plt.title(exp.experiment_name)


C:\Users\Roni\AppData\Local\Temp\ipykernel_18464\1446559763.py:33: RuntimeWarning:

divide by zero encountered in divide



In [70]:
# frequency calculations

from scipy import stats

from scipy.signal import spectrogram
phi_freq = mov.get_prop('freq_phi_lw_min_idx', wing_body='mean_body')

# stats.ttest_ind(rvs1, rvs2)


exp = experiments['manipulated_2023_08_09_60ms']
mov = exp.get_mov('mov101')

# Assuming phi_freq is already obtained and is a 1D array of sampled data
phi_freq = mov.get_prop('phi_rw', wing_body='wing')

# Check if phi_freq is 1D
if phi_freq.ndim > 1:
    phi_freq = phi_freq.flatten()

fs = 16000  # Sampling frequency in Hz


# Use a window of 30 * 73 = 2190 samples
nperseg = 1600*2
noverlap = nperseg // 1.5  # Typically, overlap is set to half the window length

# Compute the spectrogram
f, t, Sxx = spectrogram(phi_freq.T, fs=fs, nperseg=nperseg, noverlap=noverlap)

# Plot the spectrogram
plt.figure(figsize=(10, 6))
plt.pcolormesh(t, f, Sxx, shading='gouraud')
plt.ylabel('Frequency [Hz]')
plt.xlabel('Time [sec]')
plt.title('Spectrogram')
plt.colorbar(label='Intensity [dB]')
plt.show()

In [17]:

fig = go.Figure()
prop = 'vel_norm'
[exp.plot_prop_movies(prop,'body',fig,case = 'plot_exp',prop_x = 'time',color = color_map[idx],legend = exp.pertubation_name) for idx,exp in enumerate(experiments.values())]
fig.show()


fig = go.Figure()
prop = 'acc_norm'
[exp.plot_prop_movies(prop,'body',fig,case = 'plot_exp',prop_x = 'time',color = color_map[idx],legend = exp.pertubation_name) for idx,exp in enumerate(experiments.values())]
fig.show()



In [77]:
# curvture calculcations
def circumcenter(A, B, C):
    # Calculate the circumcenter and radius of the circle passing through p1, p2, p3

    D = np.cross(B-A,C-A)
    b = np.linalg.norm(A-C)
    c = np.linalg.norm(A-B)
    a = np.linalg.norm(B-C)
    R = a*b*c/2/np.linalg.norm(D)
    return R
    
# Example usage:
cm = np.hstack([exp.get_mov(mov_name).get_prop('CM_real_x_body_smth',wing_body='body',three_col=3) for mov_name in exp.mov_names])


np.savetxt('data.csv', (cm[:,0], cm[:,1], cm[:,2]), delimiter=',')

R = [circumcenter(cm[i],cm[i-1],cm[i + 1]) for i in range(1,cm.shape[0] -1 )]
print("Radius of curvature:", R)
plt.plot(R)

Radius of curvature: [18.1235062147534, 18.160533066862545, 18.19766222474049, 18.234893778461128, 18.272227804501945, 18.309664386599998, 18.34720361708127, 18.384845629941896, 18.42259038418517, 18.46043806025328, 18.498388677224476, 18.536442284848345, 18.574598983718438, 18.612858811883893, 18.65122176116999, 18.68968793447878, 18.728257380021002, 18.766930076820444, 18.80570605753329, 18.844585402454705, 18.883568050303435, 18.92265404380452, 18.961843410931724, 19.001136067640566, 19.040532135661195, 19.080031454283105, 19.119634078810943, 19.159339994292747, 19.199149151823654, 19.239061413484077, 19.27907692249847, 19.319195463626123, 19.359416998951616, 19.39974151765255, 19.440168896166696, 19.480699067809983, 19.521331888800177, 19.56206735065088, 19.60290526243713, 19.643845520832038, 19.68488805031494, 19.726032644168125, 19.76727920753462, 19.80862758749297, 19.85007757474108, 19.89162905897535, 19.93328185022444, 19.975035702394916, 20.016890486279983, 20.058845982010844

In [36]:
exp_list = []
for exp in experiments.values():
    prop = 'time'
    time_list = [exp.get_mov(mov).get_prop(prop,wing_body='body') for mov in exp.mov_names]

    prop = 'pitch_body'
    prop_list = [exp.get_mov(mov).get_prop(prop,wing_body='body') for mov in exp.mov_names]

    time_min_max = [np.min(np.vstack(time_list)),np.max(np.vstack(time_list))]

    mean_prop = []
    for t in np.arange(time_min_max[0],time_min_max[1],1/16):
        j = 0 
        for idx,tm in enumerate(time_list):
            j = 0
            prop_mean = 0
            if t in tm: 
                prop_mean += (prop_list[idx][t==tm])
                j += 1
        if j > 0:
            mean_prop.append(prop_mean/j)
        else:
            mean_prop.append(np.nan)
#     plt.plot(np.arange(time_min_max[0],time_min_max[1],1/16),mean_prop,label = exp.experiment_name)
# plt.legend()




# mean_prop

In [386]:
# הפרש בין מהירות זד בזמן כלשהו ביחס למהירות זד בזמן 0 
t0 = 0
t1 = 80
hist_plus = []
hist_minus = []


plt.figure()
for mov_name in exp.mov_names:
    mov = exp.get_mov(mov_name)
    cm_z_dot = mov.get_prop('CM_real_z_body_dot_smth','body')

    time_lst = mov.t0_t1_idx(t0,t1)
    if cm_z_dot[mov.ref_frame]> 0: 
        hist_plus.append(cm_z_dot[time_lst[1]] - cm_z_dot[mov.ref_frame])
    else: 
        hist_minus.append(cm_z_dot[time_lst[1]] - cm_z_dot[mov.ref_frame])
    
plt.hist([np.hstack(hist_plus),np.hstack(hist_minus)],stacked = True,alpha = 0.5)

plt.show()



In [379]:
exp = experiments['manipulated_2023_08_09_60ms']
delta_velocity = []
delta_v =[]
delta_ang = 70
for mov_name in exp.mov_names:
    mov = exp.get_mov(mov_name)
    delta_velocity = mov.get_prop('delta_ang_CM_dot_x',wing_body='body')
    time_prop = mov.get_prop('time',wing_body='body')

    if 200 < max(time_prop):
        t0 = 200
        t1 = -1
        time_lst = mov.t0_t1_idx(t0,t1)


        delta_v.append(delta_velocity[time_lst[0],:])
delta_v = np.array(delta_v)
delta_v[delta_v > 360] = 360 - delta_v[delta_v > 360]
delta_v[delta_v < 360] = 360 - delta_v[delta_v < 360]

print(np.sum((np.abs(delta_v) < delta_ang) | ( np.abs(360-delta_v) < delta_ang) | ( np.abs(-360+delta_v) < delta_ang)))
len(exp.mov_names)



41


83

In [19]:
exp = experiments['manipulated_11_8_23_blink40ms']
spec = []
freq = []
for mov_name in exp.mov_names:
    mov = exp.get_mov(mov_name)
    signal  = mov.get_prop('roll_body',wing_body='body').T[0]

    sampling_rate = 16000
    # Step 2: Compute the FFT
    fft_result = np.fft.fft(signal)
    fft_freq = np.fft.fftfreq(len(signal), 1/sampling_rate)

    # Step 3: Compute the amplitude spectrum
    amplitude_spectrum = np.abs(fft_result)
    spec.append(amplitude_spectrum)
    freq.append(fft_freq)

    
    # np.mean(spec)
    # Step 4: Plot the amplitude spectrum
    # plt.figure(figsize=(10, 6))
spec_freq = np.vstack((np.hstack(spec),np.hstack(freq)))
spec_freq[spec_freq[:, 1].argsort()]


plt.stem(spec_freq[1,:], spec_freq[0,:])
plt.title('Amplitude Spectrum')
plt.xlabel('Frequency (Hz)')
plt.ylabel('Amplitude')
plt.xlim(0, sampling_rate / 2)  # Focus on positive frequencies
plt.grid(True)
plt.show()